# Norms, conditions numbers and Eigensystems

In linear-algebra calculations, we are sometimes very unfortunate and have to solve a problem like $Ax = b$ (given fixed $A$), where small changes in $b$ produce extremely large changes in $x$. Such problems are said to be **ill-conditioned**.

This notebook explores this phenomenon. Along the way we will have to calculate condition numbers and eigenvalues.

## Preliminaries

Let's load numpy as usual, and the linear algebra package from numpy as we will find some functions in it useful. We also use the `GaussianElimination()` from one of the other notebooks and define the $L_2$-norm

In [55]:
import numpy as np
from numpy import linalg as la

In [56]:
def GaussianElimination(A, b):
    n = A.shape[1]
    
    # Append the vector b as a column to the matrix A
    A1 = np.c_[A,b]
    
    i = 0
    while i < n - 1:
        j = i+1
        while j < n:
            A1[j, i+1:] = A1[j, i+1:] - A1[i, i+1:]*A1[j,i]/A1[i,i]
            j += 1
        i += 1
        
    x = np.zeros(n)
    
    i = n-1
    while i >= 0:
        j = i
        x[i] = A1[i,n]
        while j < n-1:
            x[i] -= A1[i,j+1]*x[j+1]
            j += 1
        x[i] = x[i]/A1[i,i]
        i -= 1
    
    return x

In [73]:
def MatrixInverseViaGaussianElimination(A):
    n = A.shape[1]
    
    A1 = np.hstack((A,np.identity(n)))
    
    i = 0
    while i < n:
        j = 0
        while j < n:
            if(j == i): 
                j += 1
                continue
            A1[j] = (A1[j] - A1[i]*A1[j,i]/A1[i,i])
            A1[j] = A1[j]/A1[j,j]
            j += 1
        i += 1
    
    return A1[:,n:2*n]

In [57]:
def L2Norm(v):
    return la.norm(v)

## Example case

Let's look at an example where the matrix $A$ is ill-conditioned

In [58]:
A = np.array([[1.002,1],[1,0.998]])
b = np.array([2.002,1.998])

x = GaussianElimination(A,b)
print(x)

[1. 1.]


Slightly perturbing $b$ causes a huge change in the value of $x$

In [60]:
bp = np.array([2.0021,1.998])

xp = GaussianElimination(A,bp)
print(xp)

[-23.95  26.  ]


In [61]:
print("Change in b = %.4f%%" % (100*L2Norm(bp-b)/L2Norm(b)))
print("Change in x = %.2f%%" % (100*L2Norm(xp-x)/L2Norm(x)))

Change in b = 0.0035%
Change in x = 2497.50%


## Maximum eigenvalue via the power method

Also check out this [nice video](https://www.youtube.com/watch?v=yBiQh1vsCLU) showing the method in action

In [148]:
A = np.array([[-2,-2,3.],[-10,-1,6],[10,-2,-9]])

la.eig(A)

(array([  3., -12.,  -3.]), array([[ 0.57735027, -0.33333333,  0.40824829],
        [-0.57735027, -0.66666667,  0.81649658],
        [ 0.57735027,  0.66666667,  0.40824829]]))

In [214]:
def MaxEigenvalue(A, err):
    x = np.random.rand(3)
    
    n = 0.1
    nprev = 1
    while np.abs(1-n/nprev) > err:
        p = np.dot(A,x)
        nprev = n
        n = np.max(p)
        x = p/n
        
    return n

In [221]:
MaxEigenvalue(A, 1e-8)

12.000000009430828